# 과업
1. FVC 7개의 train - test 
    - 2000 DB2 a, b - size 동일 -> 우선 2000 DB2 이 데이터 셋 하나만 진행해보세요
    - DB subset간 사이즈 다른 점 유의 -> pad 상수대신에 비율 주고, 마무리 전에 이미지 사이즈 변경 resize 코드 추가 (익숙해지시면)
2. FVC - center 만 진행 
    - centershift - non crop
    - centershift - center crop
* (coreshift - 김지원 진행 예정)

## DataBase  설정
---
- FVC2000 Database2 사용 (256 × 364)
- Genuin 쌍 2800개 (8C2 X 100)  
- Imposter 쌍 4950개 (100C99) ?????
----

- FVC2000 Database4 사용 (240 x 320)
- Genuin 쌍 2800개 (8C2 X 100)  
---

- FVC2002 Database1 사용 (384 × 374)
- Genuin 쌍 2800개 (8C2 X 100)  
---

- FVC2002 Database3 사용 (300 x 300)
- Genuin 쌍 2800개 (8C2 X 100)  
---

- FVC2002 Database4 사용 (268 x 384)
- Genuin 쌍 2800개 (8C2 X 100)  
---

- FVC2004 Database2 사용 (328 x 364)
- Genuin 쌍 2800개 (8C2 X 100)  
---

- FVC2004 Database4 사용 (288 x 284)
- Genuin 쌍 2800개 (8C2 X 100)  
---
@ Total : (학습) 5600 || (테스트) 560

In [1]:
import numpy as np
import cv2
import cv2 as cv
import matplotlib.pyplot as plt
import os
from os.path import join
from skimage import io, filters, morphology, exposure
from skimage.morphology import medial_axis
import random

# Setting
- data의 root 폴더 위치

In [2]:
fing = 'fvc'

#image parse in directory
ROOT_DIR = '/media/data2/jiwon/finger/'
root_dir_path = join(ROOT_DIR, '현민님 폴더의 FVC 데이터 위치')
root_dir = sorted(os.listdir(root_dir_path))
print(root_dir[:10])


# 기본: center_shift
# 추가: center_shift_crop
type_idx = 0
aug_types = ['center_shift', 'center_shift_crop'] # 'core_shift', 'core_shift_crop' 는 김지원이 진행
aug_type = aug_types[type_idx]

# 주의 fvc의 경우 데이터 셋마다 이미지 사이즈가 다르기 때문에 현재 코드에서 ratio로 수정해야 할 것
if 'crop' in aug_type:
    crop=256
    aug_type = f'{aug_type}_{crop}'
else: 
    crop = 0

FileNotFoundError: [Errno 2] No such file or directory: '/media/data2/jiwon/finger/현민님 폴더의 FVC 데이터 위치'

# 자잘한 이미지 저장 및 이미지 plot 등의  method by openCV
You have to save in a storage if image precessing done.

In [ ]:
def save(keyPath, file_name, cv_img, rate, type):
    
    '''
    save method need to save before image preprocessing.
    It has five arguments and requirement all.
    
    keyPath is root path of original image.
    file_name is original image file name
    cv_img is whole signal of the image
    rate is for scale value
    
    '''
    if os.path.isdir(keyPath) != True:
        os.mkdir(keyPath)
    
    saved_name = os.path.join(keyPath,"{}{}.{}".format(file_name.split('.')[0], type, 'jpg'))
    # print(saved_name)
    cv.imwrite(saved_name, cv_img)
    
def plot_images(*images, sz = 8):
    images = list(images)
    n = len(images)
    fig, ax = plt.subplots(ncols=n, sharey=True, figsize = (sz, sz))
    for i, img in enumerate(images):
        ax[i].imshow(img, cmap='gray')
        ax[i].axis('off')
    plt.subplots_adjust(left=0.03, bottom=0.03, right=0.97, top=0.97)
    plt.show()

# SegmneFingerprint 
- 노이즈 제거 1
- 지문 평탄화
- 노이즈 제거 2

In [ ]:
class SegmentFingerprint():
    def __init__(self, finger, g_bl=5, m_bl=25, th=240, angle=0, b_br=55):
        self.finger = finger
        rgb = self.finger[1,1]
        self.border_color = tuple(map(int, rgb))
        self.rotated = self.rotate_image(angle)
        self.img_mask = cv2.cvtColor(self.rotated, cv2.COLOR_BGR2GRAY)
        gauss_blur = cv2.GaussianBlur(self.img_mask,(g_bl, g_bl),0)
        img_mask_blur = cv2.medianBlur(gauss_blur,m_bl)
        img_mask_blur = cv2.medianBlur(img_mask_blur,m_bl)
        ret, self.thresh_cv = cv2.threshold(img_mask_blur, th, 255, cv2.THRESH_BINARY) 
        
        self.b_br = b_br

    def rotate_image(self, angle):
        # 이미지의 중심점을 구합니다.
        height, width = self.finger.shape[:2]
        center = (width / 2, height / 2)
        # 회전 변환 행렬을 생성합니다.
        matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        # 이미지를 회전시킵니다.
        rotated = cv2.warpAffine(self.finger, matrix, (width, height), borderValue=self.border_color)
        if angle == 0: return self.finger
        return rotated
    
    def getSegmentation(self, m = 0):
        t = np.where(self.thresh_cv==0)
        self.h, self.w = self.thresh_cv.shape
        
        ymin, ymax, xmin, xmax = max(np.min(t[0][:])-m, 0), np.max(t[0])+m+1, np.min(t[1][:])-m, np.max(t[1][:])+m+1
        self.segmentation_mask = self.thresh_cv[ymin:ymax, xmin:xmax]
        self.segmentation = self.img_mask[ymin:ymax, xmin:xmax]
        self.a = self.segmentation.copy()
        self.a= cv2.copyMakeBorder(self.a, ymin, self.h-ymax, xmin, self.w-xmax, cv2.BORDER_CONSTANT, value=255)
        
    def getBinaryThreshold(self, blk_sz=17, C=16):
        self.th3 = cv2.adaptiveThreshold(self.a, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, blk_sz, C)
        
    def equalize(self, blur = 9, pad = 50):
        gauss_blur = cv2.GaussianBlur(self.th3,(blur,blur),0)  #GaussianBlur
        hist, bins = np.histogram(gauss_blur.flatten()*2, 256, [0,256]) #gauss_blur
        cdf = hist.cumsum()
        cdf_m = np.ma.masked_equal(cdf,0)
        #History Equalization 공식
        cdf_m = ((cdf_m - cdf.min())*255/(cdf_m.max()-cdf.min()))
        # Mask처리를 했던 부분을 다시 0으로 변환
        cdf = np.ma.filled(cdf_m,0).astype('uint8')
        self.equalization = cdf[self.th3]
        
        self.img_eq = exposure.equalize_hist(self.a)
        
    def getMasking(self, thr_p=0.3, blur2=25):
        # Calculate the local gradient (using Sobel filters)
        self.gx, self.gy = cv2.Sobel(self.equalization, cv2.CV_32F, 1, 0), cv2.Sobel(self.equalization, cv2.CV_32F, 0, 1)
        self.gx2, self.gy2 = self.gx**2, self.gy**2
        self.gm = np.sqrt(self.gx2 + self.gy2)

        self.sum_gm = cv2.boxFilter(self.gm, -1, (self.b_br, self.b_br), normalize = False)

        # Use a simple threshold for segmenting the fingerprint pattern
        self.thr = self.sum_gm.max() * thr_p
        self.mask = cv2.threshold(self.sum_gm, self.thr, 255, cv2.THRESH_BINARY)[1].astype(np.uint8)

        # 구조화 요소 커널, 사각형 (3x3) 생성 ---①
        self.k = cv2.getStructuringElement(cv2.MORPH_RECT, (blur2, blur2))
        # 침식 연산 적용 ---②
        erod = cv2.erode(self.mask, self.k)
        self.mask = cv2.erode(erod, self.k)

        self.not_mask = cv2.bitwise_not(self.mask)
        self.final_seg = np.bitwise_or(self.not_mask, self.equalization)


# Core 탐지를 위한 코드 1 - MiniFingerprint
- 현재 부탁드린 업무에서는 사용되지 않습니다.
- 추후 minutia 코드에서는 사용될 수 있습니다.

# 세선화 코드 - thinning_func
- 현재 부탁드린 업무에서는 사용되지 않습니다.
- 추후 minutia 코드에서는 사용될 수 있습니다.

In [ ]:
class MiniFingerprint():
    def __init__(self, original_print, unsegemented_print, original_mask, padding=20, mini_size=224, blur = 70):
        self.orig_print = original_print.copy()
        self.unseg_print = unsegemented_print.copy()
        self.orig_mask = original_mask
        self.h, self.w = self.orig_print.shape
        self.p_sz = padding
        self.m_sz = mini_size
        self.W = (blur, blur)
        
        self.mini_print = cv2.resize(self.orig_print, (self.m_sz, self.m_sz))#[10:-10, 10:-10]
        self.mini_mask = cv2.resize(self.orig_mask, (self.m_sz, self.m_sz))#[10:-10, 10:-10]
        
    
    def getCoreMap(self, n=10):
        temp_int_ori = self.int_mini_ori.astype(float)
        self.temp_int_ori = cv2.resize(temp_int_ori, (self.w, self.h))
        self.temp_ori = cv2.resize(self.mini_ori, (self.w, self.h))
        self.core_ori = self.temp_ori.copy()
        self.y, self.x = np.unravel_index(np.argmax(self.core_ori), self.core_ori.shape)
        
        cv2.circle(self.core_ori, (self.x,self.y), n*2, (0, 100, 10), n)
        self.getCoreFingerprint(n=n)
        
    def getAdaptiveCore(self, n=10):
        miniOri = self.int_mini_ori.astype(float) #= mini_FP.getOrientationMap()
        self.mini_gogo = cv2.inRange(miniOri, 4.5, 5.0)
        self.mask_gogo = cv2.resize(self.mini_gogo, (self.w, self.h))
        self.temp_gogo = cv2.resize(miniOri, (self.w, self.h))
        
        self.continueMiniOri = self.unmasked_mini_ori.astype(float) #= mini_FP.getOrientationMap()
        self.continueMiniOri = cv2.resize(self.continueMiniOri, (self.w, self.h))
        self.ths_ori = cv2.normalize(self.continueMiniOri, None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8U)
        self.ths_ori = cv2.adaptiveThreshold(self.ths_ori,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,15,4)
        self.ths_ori[self.orig_mask == 0] = 0
        self.continueMiniOri[self.orig_mask == 0] = 0
        
        self.core_gogo = self.temp_gogo.copy()
        self.y, self.x = np.unravel_index(np.argmax(self.ths_ori), self.ths_ori.shape)
        cv2.circle(self.core_gogo, (self.x,self.y), n*2, (0, 100, 10), n)
        self.getCoreFingerprint(n=n)
        
    def getCoreFingerprint(self, n=10):
        
        cv2.circle(self.orig_print, (self.x,self.y), n, (0, 100, 10), n)
        cv2.circle(self.unseg_print, (self.x,self.y), n, (0, 100, 10), n)
        
    def changeCoreCoordinate(self, x, y):
        self.y = y
        self.x = x
        
    def getOrientationMap(self):
        self.x_g = cv2.Sobel(self.mini_print, cv2.CV_32F, 1, 0)
        self.y_g = cv2.Sobel(self.mini_print, cv2.CV_32F, 0, 1)
        self.x_g[self.mini_mask==0] = 0
        self.y_g[self.mini_mask==0] = 0
        self.x2_g = self.x_g**2
        self.y2_g = self.y_g**2 
        
        
        self.xx_g = cv2.boxFilter(self.x2_g, -1, self.W, normalize = False)
        self.yy_g = cv2.boxFilter(self.y2_g, -1, self.W, normalize = False)
        self.x_y_g = self.xx_g - self.yy_g # gxx_gyy = gxx - gyy
        
        self.xy_g = cv2.boxFilter(self.x_g * self.y_g, -1, self.W, normalize = False)
        self.xy2_g = 2 * self.xy_g # gxy2 = 2 * gxy
        self.unmasked_mini_ori = (cv2.phase(self.x_y_g, self.xy2_g) + np.pi) / 2
        # '-' to adjust for y axis direction
        
        self.mini_ori = self.unmasked_mini_ori.copy()
        self.mini_ori[self.mini_mask == 0] = 0

        self.int_mini_ori = np.round(self.mini_ori)
    
        return self.int_mini_ori
    
def thinning_func(img):
    skel, distance = medial_axis(img, return_distance=True)
    dist_on_skel = (distance * skel).astype(np.uint8)
    dist_on_skel[dist_on_skel != 0] = 255
    return dist_on_skel

# Data augmentation 
1. data_flip 
- 모르셔도 됩니다.
- 수직 수평으로 뒤집을 수 있습니다.

2. data_rotate
- 이미지를 회전합니다.
- 회전 방법은 아시면 도움이 되나, 현재 회전 단독만 사용하진 않아서, 해당 코드도 넘어가셔도 됩니다.

In [ ]:
def data_flip(saved_dir, data, img, rate, type, saving_enable=False):
    
    img = cv.flip(img, type)
    try:
        if type == 0:
            type = "_horizen_"
        elif type == 1:
            type = "_vertical_"
        elif type == -1:
            type = "_bothFlip_"
        
        if saving_enable == True:
            save(saved_dir, data, img, rate, type)
            return img
    
    except Exception as e:
        print(e)
        return "Failed"

def data_rotate(saved_dir, data, img, rate, type, saving_enable=False):

    # Generate a random angle within the specified range
    rate = random.uniform(-rate, rate)

    xLength = img.shape[0]
    yLength = img.shape[1]
    
    try:
        rotation_matrix = cv.getRotationMatrix2D((xLength/2 , yLength/2), rate, 1)
        img = cv.warpAffine(img, rotation_matrix, (xLength, yLength), borderMode=cv.BORDER_CONSTANT, borderValue=[255, 255, 255])
        #print(img.shape)        
        if saving_enable == True:
            save(saved_dir, data, img, rate, type)
        
        return "Success"
    except Exception as e:
        print(e)
        return "Failed"

# 중요 데이터 전처리 코드
1. data_center_crop
 - 원하는 사이즈를 전달하면, 해당 이미지의 중심을 사용해서 모서리 부근의 이미지를 잘라냅니다.

 2. data_rotate_correct_skew
 - 이미지의 마스크에 컨투어를 이용하여 지문을 감싸는 사각형을 만든다.
 - 해당 사각형의 중점을 활용해서 위치 이동

In [ ]:
def data_center_crop(img, set_size):
    h,w = img.shape[:2]
    
    if set_size > min(h,w):
        return img
    crop_width = set_size
    crop_height = set_size
    
    mid_x, mid_y = w//2, h//2
    offset_x, offset_y = crop_width//2, crop_height//2
    
    crop_img = img[mid_y - offset_y:mid_y +offset_y, mid_x - offset_x:mid_x +offset_x]
    return crop_img

def data_rotate_correct_skew(saved_dir, data, img, rate, type, saving_enable=False, crop = 0, pad = 50): # pad는 padding 즉 마진 크기 -> ratio로 수정하시면 좋음

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = 255 - gray
    
    # Rotate image to deskew
    (h, w) = img.shape[:2]
    
    # 이미지 사이즈나 선명도를 기준으로 필터 크기 조절하기 가능
    # g_bl : 가우시안 블러 사이즈
    # m_bl : 미디안 블러 사이즈
    # th   : 바이너리 임계치
    # angle: 회전 각도 - 사용하지 않음
    # b_br : 최종 마스킹 단계의 블러 사이즈
    seg_FP = SegmentFingerprint(finger=img, g_bl=5, m_bl=25, th=240, angle=0, b_br=55)
    # 1. 노이즈 제거용 첫번째 마스크 적용
    seg_FP.getSegmentation(m=0)
    # 2. 흑백 바이너리 이미지로 변경
    seg_FP.getBinaryThreshold(blk_sz=17, C=10)
    # 3. 이미지 평탄화 작업 진행
    seg_FP.equalize(pad=pad)
    # 4. 최종 마스킹 적용 - 소벨 필터와 모폴로지 기법을 응용함
    seg_FP.getMasking()
    # 5. 지문 영역 분할용 마스크 도출
    img_m = seg_FP.mask
    # 5-1. 마스크에 마진 추가하여 이미지의 배경 부분 확대 - 컨투어 과정에서 사용
    img_m= cv2.copyMakeBorder(img_m, pad, pad, pad, pad, cv2.BORDER_CONSTANT, value=0)
    contours, _ = cv2.findContours(img_m, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #RETR_EXTERNAL
    
    # 6. 컨투어 후보 중에 불필요한 컨투더들 제거
    max, maxid = 0, 0
    cc = 0
    for i in range(len(contours)):
        pts = contours[i]
        if cv2.contourArea(pts) < 30000: # 영역 계산, 노이즈 제거
            continue
        # if cv2.contourArea(pts) > 100000: # 영역 계산, 노이즈 제거
        #     continue
        
        cnt = contours[i]
        rc = cv2.boundingRect(pts) # 외곽선 좌표 입력하면 사각형 좌표 반환
        mmt = cv2.moments(cnt)
        x_c, y_c = int(mmt['m10']/mmt['m00']), int(mmt['m01']/mmt['m00'])
        # img_m를 저장하거나 출력해서 결과물의 이동량 알 수 있음 
        cv2.putText(img_m, "{:.2f} M ".format(cv2.contourArea(cnt))+f"{len(contours), i, cc}", (x_c, y_c), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2)
        cv2.putText(img_m, "{:.2f} M ".format(cv2.contourArea(cnt))+f"{len(contours), i, cc}", (x_c, y_c+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        
        if max < cv2.contourArea(pts) :
            # premax = max
            # premaxid = maxid
            max = cv2.contourArea(pts)
            maxid = i
            cc += 1
            
    cnt = contours[maxid]
    
    # cv2.putText(img, "- {:.2f}".format(cv2.contourArea(cnt)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    rc = cv2.boundingRect(cnt) # 외곽선 좌표 입력하면 사각형 좌표 반환
    cv2.rectangle(img_m, rc, (255, 0, 0), 10)
    cv2.rectangle(img_m, rc, (0, 0, 0), 4)
    mmt = cv2.moments(cnt)
    # x_coordinates = coordinates[0]
    # y_coordinates = coordinates[1]

    # x_min, x_max, y_min, y_max = np.min(x_coordinates), np.max(x_coordinates), np.min(y_coordinates), np.max(y_coordinates)

    # x_c, y_c = int((x_max + x_min) / 2), int((y_max + y_min) / 2)
    x_c, y_c = int(mmt['m10']/mmt['m00']), int(mmt['m01']/mmt['m00'])
    print(x_c, y_c)
    cv2.putText(img_m, "{:.2f} M ".format(cv2.contourArea(cnt))+f"{len(contours), maxid, cc}", (x_c, y_c), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2)
    cv2.putText(img_m, "{:.2f} M ".format(cv2.contourArea(cnt))+f"{len(contours), maxid, cc}", (x_c, y_c+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    # cv2.circle(seg_FP.not_mask, (x_c, y_c), 10*2, (0, 100, 10), 10)
    cv2.circle(img_m, (x_c, y_c), 10*2, (255, 255, 255), 10)
    cv2.circle(img_m, (x_c, y_c), 7*2, (0, 0, 0), 10)
    x_c, y_c = x_c-pad, y_c-pad
    
    
    w_gap = ((w // 2) - x_c)
    h_gap = ((h // 2) - y_c)
    
    coords = np.column_stack(np.where(seg_FP.mask > 0))
    angle0 = cv2.minAreaRect(coords)[-1]
    
    if angle0 < -45:
        angle = 90 + angle0
    elif angle0 > 84:
        angle = -90+angle0
    elif angle0 > 45:
        angle = 90 -angle0
    else :
        angle = angle0
    
    try:
        img_m = img_m[pad:-pad, pad:-pad]
        (h_m, w_m) = img_m.shape[:2]
        if 'shifted' in saved_dir:
            # 변환 행렬, X축으로 10, Y축으로 20 이동
            M = np.float32([[1,0,w_gap],[0,1,h_gap]])
            img_m = cv2.warpAffine(img_m, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv.BORDER_CONSTANT, borderValue=[0, 0, 0])
            img = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv.BORDER_CONSTANT, borderValue=[255,255,255])
            
        if crop > 0 :   img = data_center_crop(img, crop) # crop = 512
        
        # M = cv2.getRotationMatrix2D(center, angle, 1.0)
        # rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
        # img_m = cv2.warpAffine(img_m, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv.BORDER_CONSTANT, borderValue=[0, 0, 0])
        # img = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv.BORDER_CONSTANT, borderValue=[255,255,255])
        # cv2.putText(img_m,f"mask {h_m, w_m} ori {h,w}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,255), 2)
        #print(img.shape)        
        if saving_enable == True:
            # cv2.putText(rotated, "Original: {:.2f} => Rotate".format(angle0) +f"{angle}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            save(saved_dir, data, img, rate, type)
            # save(saved_dir, data, img_m, rate, "_mask")
            # cv2.putText(seg_img, "Original: {:.2f} => Rotate".format(angle0) +f"{angle}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            # save(saved_dir, data, seg_img, rate, type+"_mask")
        return "Success"
    except Exception as e:
        print(e)
        return "Failed"

In [ ]:
def augmente(keyName, rate=None, if_scale=False, save_dir='새롭게 저장하고 싶으신 위치', crop=0):
    
    keyPath = os.path.join(root_dir_path, keyName) # keypath direct to root path
    datas = os.listdir(keyPath)
    data_total_num = len(datas)
    print("Overall data in {} Path :: {}".format(keyPath, data_total_num))

    try:
        for data in sorted(datas):
            data_path = os.path.join(keyPath, data)
            img = cv.imread(data_path)
            data_rotate_correct_skew(save_dir, data, img, rate=0, type="", 
                                             saving_enable=True, crop=crop)
        return "success"
    
    except Exception as e:
        print(e)
        return "Failed"

# Create Entry Point function


In [ ]:
def main_TransformImage(keyNames, save_dir = '저장하시고 싶은 위치'):
    try:
        if fing == 'fvc':
            for i in range(len(keyNames)):
                if i <= 7:
                    keyPath = sorted(keyNames)[i]
                    print(i, keyPath) 
                else: break
                
                
                if os.path.isdir(join(save_dir, aug_type+keyPath)) != True: 
                    os.mkdir(join(save_dir, aug_type+keyPath))
                augmente(keyPath, save_dir=join(save_dir, 'skew_'+keyPath), crop=crop)
                
                
                
        elif 'poly' in fing:
            
            for i in range(2): # 328*356 size 
                keyPath = sorted(keyNames)[i]
                print(i, keyPath) 
                
                if os.path.isdir(join(save_dir, aug_type+keyPath)) != True: 
                    os.mkdir(join(save_dir, aug_type+keyPath))
                augmente(keyPath, save_dir=join(save_dir, aug_type+keyPath), crop=crop)
        
        return "Augment Done!"
    except Exception as e:
        print(e)
        return "Augment Error!"

In [ ]:
main_TransformImage(keyNames=root_dir, save_dir='저장하고싶으신 위치')

0 first_session
Overall data in /media/data2/jiwon/finger/data/PolyU_Dataset/contact-based_fingerprints/first_session Path :: 2016
208 239
220 246
266 195
212 215


227 217
234 198
204 262
220 208
206 213
216 221
211 219
217 218
229 258
255 209
251 199
199 221
222 206
204 216
215 237
179 240
195 257
176 258
228 246
213 240
231 291
210 221
229 199
183 253
198 267
183 234
219 250
224 238
210 205
231 193
224 201
203 217
224 205
217 179
211 211
206 195
206 204
218 197
210 196
217 181
222 199
239 195
223 184
192 198
181 222
228 195
237 191
228 197
233 197
194 182
259 224
230 222
205 225
249 227
228 216
230 204
238 201
275 218
231 199
215 195
198 198
230 199
212 219
236 197
245 200
225 213
233 216
244 210
194 213
206 196
230 188
256 213
226 209
223 206
253 181
194 197
279 214
197 190
232 183
235 192
221 211
224 191
191 245
212 218
218 196
209 234
217 207
244 216
217 224
220 218
229 206
202 196
262 218
238 220
221 231
258 172
210 208
228 242
246 215
200 257
237 278
213 266
223 193
212 216
201 254
261 254
194 251
207 218
260 209
208 202
181 201
218 196
211 204
208 186
208 200
207 192
197 215
180 217
223 245
235 190
178 190
233 215
240 195
227 179
223 242


'Augment Done!'